# Part 1: Spark and Spark SQL

## Task2 : Webgraph on Internal Links

In [1]:
## initial Setup

import re 
import regex
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, lower
from pyspark.sql.types import StringType, ArrayType

spark = SparkSession.builder.getOrCreate()
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/30 00:38:11 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/30 00:38:11 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/30 00:38:11 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/30 00:38:11 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
## df

small = spark.read.format('xml').options(rowTag='page').load('hdfs:/enwiki_small.xml')
#test = spark.read.format('xml').options(rowTag='page').load('hdfs:/enwiki_test.xml')

22/04/30 00:38:29 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
22/04/30 00:38:29 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #0,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at c

In [3]:
def get_internal_link(text):
    matches = regex.findall(r'\[\[((?:[^[\]]+|(?R))*+)\]\]', text)
    result = []
    for match in matches:
        splited = re.split(r'\|', match)
        temp = []
        for link in splited:
            if len(re.findall(r'^category:', link)) >0:
                temp.append(link)
            elif len(re.findall(r':|#', link)) > 0:
                continue
            else:
                temp.append(link)
        if len(temp)>0 :
            result.append(temp[0]) 
        else:
            continue
    return result

get_internal_link = udf(get_internal_link, ArrayType(StringType()))

## map reduce function
def process(df):
    output = df.select(lower(df.title).alias('title'), lower(df.revision.text._Value).alias('text')) \
            .dropna(how = 'any') \
            .withColumn('internal_link', get_internal_link('text')) \
            .select('title', explode('internal_link').alias('internal_link')) \
            .sort('title', 'internal_link')
    return output

In [4]:
# small
small_result = process(small)
small_result.show()

+----------------+--------------------+
|           title|       internal_link|
+----------------+--------------------+
|"love and theft"| mississippi (bob...|
|"love and theft"|                2001|
|"love and theft"|accidents & accus...|
|"love and theft"|           accordion|
|"love and theft"|            allmusic|
|"love and theft"|   americana (music)|
|"love and theft"|anthology of amer...|
|"love and theft"|        augie meyers|
|"love and theft"|        augie meyers|
|"love and theft"|               banjo|
|"love and theft"|         bass guitar|
|"love and theft"|       billboard 200|
|"love and theft"|       billboard 200|
|"love and theft"|  blender (magazine)|
|"love and theft"|               blues|
|"love and theft"|           bob dylan|
|"love and theft"|           bob dylan|
|"love and theft"|           bob dylan|
|"love and theft"|           bob dylan|
|"love and theft"|           bob dylan|
+----------------+--------------------+
only showing top 20 rows



In [7]:
# test
test_result = process(test).limit(5)
test_result.show()

+-----------+-------------------+
|      title|      internal_link|
+-----------+-------------------+
|  ! (chess)|punctuation (chess)|
|!!! (album)|                !!!|
|!!! (album)|                !!!|
|!!! (album)|              album|
|!!! (album)|              album|
+-----------+-------------------+



In [5]:
#save samll result to gcs
gcs_filepath = 'gs://hw2-backup/p1t2.csv'
small_result.limit(5).coalesce(1).write.option("delimiter", "\t").csv(gcs_filepath)